# Multimodal Travel Planner

In [138]:
import os
import requests
import pickle 
import math
import folium
import pandas as pd
from dotenv import load_dotenv

import googlemaps

# load API keys
load_dotenv()
maps_api_key = os.getenv("MAPS_API_KEY")
amadeus_api_key = os.getenv("AMADEUS_API_KEY")
amadeus_api_secret = os.getenv("AMADEUS_API_SECRET")

gmaps = googlemaps.Client(key=maps_api_key)


In [139]:

# Load .env file
load_dotenv()

def get_amadeus_token():
    client_id = os.getenv("AMADEUS_API_KEY")
    client_secret = os.getenv("AMADEUS_API_SECRET")

    response = requests.post(
        'https://test.api.amadeus.com/v1/security/oauth2/token',
        data={
            'grant_type': 'client_credentials',
            'client_id': client_id,
            'client_secret': client_secret
        }
    )

    if response.status_code == 200:
        return response.json().get("access_token")
    else:
        print("Amadeus token request failed:", response.status_code, response.text)
        return None


In [140]:
airports = {
    "DEN": ("Denver International", (39.8561, -104.6737)),
    "SLC": ("Salt Lake City Intl", (40.7899, -111.9791)),
    "SMF": ("Sacramento Intl", (38.6951, -121.5908)),
    "LAX": ("Los Angeles Intl", (33.9416, -118.4085)),
    "DTW": ("Detroit Metro", (42.2162, -83.3554)),
    "ZRH": ("Zurich", (47.4581, 8.5555)),
    "GVA": ("Geneva", (46.2381, 6.1096)),
    "INN": ("Innsbruck", (47.2602, 11.3430)),
    "CDG": ("Paris Charles de Gaulle", (49.0097, 2.5479)),
    "MSP": ("Minneapolis-St Paul", (44.8848, -93.2223))
}

# Flat flight estimate
def estimate_flight_price(distance_miles):
    return round(0.35 * distance_miles + 50, 2)

In [141]:
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c * 0.621371

def coords_match(coord1, coord2, tolerance_miles=0.001):
    return haversine(coord1, coord2) <= tolerance_miles

def estimate_cost(mode, distance):
    if mode == "driving": return distance * 0.5
    elif mode == "transit": return distance * 0.2 + 2
    elif mode == "flight": return 50 + distance * 0.15
    return 0

def nearest_airport(location, airport_dict):
    closest = None
    min_dist = float('inf')
    for code, (name, coords) in airport_dict.items():
        dist = haversine(location, coords)
        if dist < min_dist:
            min_dist = dist
            closest = (code, (name, coords))
    return closest

In [142]:
def get_flight_price(origin_code, dest_code, departure_date="2025-12-15", adults=1):
    access_token = get_amadeus_token()
    if access_token is None:
        print("Failed to retrieve access token.")
        return None

    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    params = {
        "originLocationCode": origin_code,
        "destinationLocationCode": dest_code,
        "departureDate": departure_date,
        "adults": adults,
        "travelClass": "ECONOMY",
        "currencyCode": "USD",
        "max": 1
    }

    response = requests.get(
        "https://test.api.amadeus.com/v2/shopping/flight-offers",
        headers=headers,
        params=params
    )

    try:
        offers = response.json().get("data", [])
        if offers:
            return float(offers[0]["price"]["total"])
        else:
            print(f"No flight offers found from {origin_code} to {dest_code}.")
    except Exception as e:
        print(f"Error parsing flight price: {e}")
        print("Response JSON:", response.json())
    return None


In [143]:
get_flight_price("JFK", "LAX")

183.45

In [153]:
from collections import OrderedDict

# Load original route
path = os.path.join("Optimal_Routes", "paths_route.pkl")
with open(path, "rb") as f:
    route_dict = pickle.load(f)

# Convert to ordered list of items
route_items = list(route_dict.items())

# Grab first and last
first_location, first_coords = route_items[0]
last_location, _ = route_items[-1]

# Force append if last != first
if last_location != first_location:
    print(f"Appending {first_location} to complete round trip...")
    route_items.append((first_location, first_coords))


route_dict = OrderedDict()
for loc, coords in route_items:
    # Allow duplicates in keys by making the return leg name slightly different
    if loc in route_dict:
        route_dict[loc + " (return)"] = coords
    else:
        route_dict[loc] = coords

Appending Denver Airport to complete round trip...


In [ ]:
# store trip leg info
journey_log = []

# resort names and coords
locations = list(route_dict.keys())
location_coords = route_dict

# set origin and destination plus associated coordinates
# loop through one leg at a time
for i in range(len(locations) - 1):
    origin = locations[i]
    dest = locations[i + 1]
    origin_coords = location_coords[origin]
    dest_coords = location_coords[dest]

# string format for gmaps query
    origin_str = f"{origin_coords[0]},{origin_coords[1]}"
    dest_str = f"{dest_coords[0]},{dest_coords[1]}"

# initilize best mode, cost, and distance
    best_mode = None
    best_cost = float('inf')
    best_distance = 0

    # queries google maps directions API for transit and driving data
    for mode in ["transit", "driving"]:
        try:
            # queries directions API for a route between origin and dest using mode
            directions = gmaps.directions(origin_str, dest_str, mode=mode)
            # if route is found 
            if directions:
                # contains distance and duration of route
                leg = directions[0]['legs'][0]
                # meters to miles
                distance = leg['distance']['value'] * 0.000621371
                # estimates cost given mode and distance
                cost = estimate_cost(mode, distance)
                # ensure best cost is the lowest cost found for leg
                if cost < best_cost:
                    best_cost = cost
                    best_distance = distance
                    best_mode = mode
        # catches error fetching directions between origin and destination            
        except Exception as e:
            print(f"Error fetching {mode} directions from {origin} to {dest}: {e}")
            continue

    # flight if haversine distance is greater than 150
    direct_distance = haversine(origin_coords, dest_coords)
    # threshold = 150
    if direct_distance > 150:
        # find nearest airport to origin and destination
        try:
            o_code, (o_name, o_coords) = nearest_airport(origin_coords, airports)
            d_code, (d_name, d_coords) = nearest_airport(dest_coords, airports)
        # tries to get real flight cost from Amadeus
            try:
                flight_price = get_flight_price(o_code, d_code)
                if flight_price is None:
                    raise ValueError("No price from Amadeus")
                
            # manually estimates flight price if Amadeus query doesn't work
            except:
                flight_price = estimate_flight_price(direct_distance)

            # driving directions to airprt
            ground1 = gmaps.directions(origin_str, o_name, mode="driving")[0]['legs'][0]

            # driving directions from airport
            ground2 = gmaps.directions(d_name, dest_str, mode="driving")[0]['legs'][0]

            # total cost and distance when considering travel to and from airport
            total_flight = (
                float(flight_price) +
                estimate_cost("driving", ground1['distance']['value'] * 0.000621371) +
                estimate_cost("driving", ground2['distance']['value'] * 0.000621371)
            )

            # if flight is cheapest, add 3 legs (to, flight, from)
            if total_flight < best_cost:
                journey_log.extend([
                    # to airport sub-leg
                    {"from": origin, "to": o_name, "mode": "driving",
                     "distance_miles": round(ground1['distance']['value'] * 0.000621371, 2),
                     "cost_usd": round(estimate_cost("driving", ground1['distance']['value'] * 0.000621371), 2)},
                     # flight sub-leg
                    {"from": f"{o_code} Airport", "to": f"{d_code} Airport", "mode": "flight",
                     "distance_miles": round(direct_distance, 2),
                     "cost_usd": round(float(flight_price), 2)},
                     # from airport sub-leg
                    {"from": d_name, "to": dest, "mode": "driving",
                     "distance_miles": round(ground2['distance']['value'] * 0.000621371, 2),
                     "cost_usd": round(estimate_cost("driving", ground2['distance']['value'] * 0.000621371), 2)}
                ])
                continue
            # log possible error from flight eval
        except Exception as e:
            print(f"Flight fallback failed for {origin} to {dest}: {e}")

    # fallback if no modes work
    if best_mode is None:
        print(f"No route found from {origin} to {dest}. Setting fallback values.")
        journey_log.append({
            "from": origin,
            "to": dest,
            "mode": "unavailable",
            "distance_miles": 0.0,
            "cost_usd": float('inf')
        })
        # if driving or transit is cheapest, log single leg here
    else:
        journey_log.append({
            "from": origin,
            "to": dest,
            "mode": best_mode,
            "distance_miles": round(best_distance, 2),
            "cost_usd": round(best_cost, 2)
        })


No flight offers found from SMF to INN.


## OR-Tools Optimization

In [148]:
OR_paths_df = pd.DataFrame(journey_log)
OR_paths_df["leg"] = OR_paths_df.index + 1
print(OR_paths_df)
print(f"Total Cost: ${OR_paths_df['cost_usd'].sum():.2f}")
print(f"Total Distance: {OR_paths_df['distance_miles'].sum():.2f} mi")

                    from                       to     mode  distance_miles  \
0         Denver Airport                   Eldora  transit           67.24   
1                 Eldora                  A-Basin  transit          141.99   
2                A-Basin                 Keystone  driving            7.56   
3               Keystone             Breckenridge  driving           18.20   
4           Breckenridge                     Vail  driving           44.57   
5                   Vail             Beaver Creek  driving           22.51   
6           Beaver Creek                  Canyons  transit          728.80   
7                Canyons      Salt Lake City Intl  driving           27.41   
8            SLC Airport              SMF Airport   flight          472.85   
9        Sacramento Intl                 Kirkwood  driving           96.55   
10              Kirkwood                 Heavenly  driving           40.05   
11              Heavenly                Northstar  driving      

## Visualizing OR-Tools Optimization Trip

In [149]:
from tabulate import tabulate

OR_paths_df = OR_paths_df.iloc[:-1].reset_index(drop=True)
OR_paths_df["leg"] = OR_paths_df.index + 1  # Recalculate leg numbers


print(tabulate(
    OR_paths_df[["from", "to", "mode", "distance_miles", "cost_usd"]],
    headers="keys",
    tablefmt="grid",
    showindex=True
))

total_cost = OR_paths_df["cost_usd"].sum()
total_distance = OR_paths_df["distance_miles"].sum()

print("\n" + "="*60)
print(f"Total Cost: ${total_cost:,.2f}")
print(f"Total Distance: {total_distance:,.2f} miles")
print("="*60)

+----+---------------------+---------------------+---------+------------------+------------+
|    | from                | to                  | mode    |   distance_miles |   cost_usd |
+====+=====================+=====================+=========+==================+============+
|  0 | Denver Airport      | Eldora              | transit |            67.24 |      15.45 |
+----+---------------------+---------------------+---------+------------------+------------+
|  1 | Eldora              | A-Basin             | transit |           141.99 |      30.4  |
+----+---------------------+---------------------+---------+------------------+------------+
|  2 | A-Basin             | Keystone            | driving |             7.56 |       3.78 |
+----+---------------------+---------------------+---------+------------------+------------+
|  3 | Keystone            | Breckenridge        | driving |            18.2  |       9.1  |
+----+---------------------+---------------------+---------+----------

In [154]:
import folium
from folium.plugins import PolyLineTextPath

# Reusable function to resolve coordinates
def resolve_coord(name):
    if name in location_coords:
        return location_coords[name]
    for code, (label, coord) in airports.items():
        if name in [label, f"{code} Airport"]:
            return coord
    return None

# Icon dictionary
mode_icons = {
    "driving": "car",
    "transit": "train",
    "flight": "plane"
}

# Create base map
m = folium.Map(location=resolve_coord("Denver Airport"), zoom_start=5)

# Iterate through journey legs with direction, order, and icons
for i, leg in enumerate(journey_log):
    leg_num = i + 1
    start = leg["from"]
    end = leg["to"]
    mode = leg["mode"]

    start_coord = resolve_coord(start)
    end_coord = resolve_coord(end)

    if start_coord and end_coord:
        # Add markers with leg number in tooltip
        folium.Marker(
            location=start_coord,
            tooltip=f"{leg_num}: {start}",
            icon=folium.Icon(color="blue", icon=mode_icons.get(mode, "info-sign"), prefix="fa")
        ).add_to(m)

        folium.Marker(
            location=end_coord,
            tooltip=f"{leg_num}: {end}",
            icon=folium.Icon(color="green", icon=mode_icons.get(mode, "info-sign"), prefix="fa")
        ).add_to(m)

        # Add directional line with arrow
        line = folium.PolyLine(
            locations=[start_coord, end_coord],
            color="black" if mode == "flight" else "blue",
            weight=3,
            opacity=0.8
        ).add_to(m)

        # Arrows to show direction
        PolyLineTextPath(
            line,
            '➔',
            repeat=True,
            offset=7,
            attributes={'fill': 'black', 'font-weight': 'bold', 'font-size': '16'}
        ).add_to(m)

# Save and display map
save_path = os.path.join("Maps", "OR_tools_trip_map.html")
m.save(save_path)
m


## Open TSP Optimization

## Visualizing Open TSP Optimization

## Cost Optimization

## Visualizing Cost Optimization